# KNeighborsTimeSeriesRegressor
Distance-based regression predicts a target by averaging the outputs of the most similar time series.

## Regression rule
Given $k$ nearest neighbors $\mathcal{N}_k(x)$, a simple predictor is

$$\hat{y} = \frac{1}{k} \sum_{i \in \mathcal{N}_k(x)} y_i$$

Distances can be Euclidean, DTW, or other elastic measures.

## Toy dataset: sine waves with varying amplitude

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go

rng = np.random.default_rng(123)

def make_series(amplitude, length=60, noise=0.15):
    t = np.arange(length)
    y = amplitude * np.sin(2 * np.pi * t / length) + rng.normal(0, noise, size=length)
    return pd.Series(y)

amplitudes = np.linspace(0.5, 2.5, 50)
X = pd.DataFrame({"ts": [make_series(a) for a in amplitudes]})
y = amplitudes

fig = go.Figure()
for i in [0, 10, 25, 40]:
    fig.add_trace(go.Scatter(y=X.iloc[i, 0], name=f"amp={y[i]:.2f}"))
fig.update_layout(title="Sample series with different amplitudes")
fig.show()

## Fit with sktime

In [ ]:
from sktime.regression.distance_based import KNeighborsTimeSeriesRegressor

X_train, X_test = X.iloc[:40], X.iloc[40:]
y_train, y_test = y[:40], y[40:]

reg = KNeighborsTimeSeriesRegressor(n_neighbors=3, distance="dtw")
reg.fit(X_train, y_train)
pred = reg.predict(X_test)

import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test, y=pred, mode="markers", name="pred vs true"))
fig.add_trace(go.Scatter(x=y_test, y=y_test, mode="lines", name="ideal", line=dict(dash="dash")))
fig.update_layout(title="KNN regressor predictions", xaxis_title="true amplitude", yaxis_title="predicted")
fig.show()

## Notes
- Normalize series before distance calculations if scale varies widely.
- DTW handles phase shifts but is slower than Euclidean distance.
- Keep $k$ small to preserve locality; cross-validate for stability.